In [11]:
import pandas as pd
import numpy as np
import re
# Filter all warnings. 
import warnings
warnings.filterwarnings('ignore')

# Function to clean the names 
def Clean_names(Country_name): 
    # Search for opening bracket in the name followed by 
    # any characters repeated any number of times 
    if re.search('\(.*', Country_name): 
        # Extract the position of beginning of pattern 
        pos = re.search('\(.*', Country_name).start() 
        # return the cleaned name 
        return Country_name[:pos] 
    elif re.search('\d', Country_name): 
        # Extract the position of beginning of pattern 
        pos = re.search('\d', Country_name).start() 
        # return the cleaned name 
        return Country_name[:pos]
    else: 
        # if clean up needed return the same name 
        return Country_name 

# Then I will upload the excel dataset and clean it a bit to get rid of the useless columns and rows
Energy=pd.read_excel('assets/Energy Indicators.xls', header=16)
Energy=Energy[:228].drop([0]).drop([Energy.columns[0] , Energy.columns[1]] ,  axis='columns')

#Next, I'll change some of the labes for readability
Energy.rename(columns={'Unnamed: 2': 'Country', 'Renewable Electricity Production': '% Renewable'}, inplace=True)
#change some of the missing data to NaN values using lambda functions
Energy['Energy Supply']=Energy['Energy Supply'].apply(lambda x: np.nan if type(x)!=int else x)
Energy['Energy Supply per capita']=Energy['Energy Supply per capita'].apply(lambda x: np.nan if type(x)!=int else x)
#and then converting petabytes to gigabytes
Energy['Energy Supply']=Energy['Energy Supply']*1000000

#We apply the cleaning function so we only get the names, without special characters or numbers
Energy['Country'] = Energy['Country'].apply(Clean_names) 
#Then we rename some of the countries that will be used later on (I guess there's a better way, but idk how right now)  
Energy.set_index('Country', inplace=True)
Energy.rename(index={"Republic of Korea": "South Korea",
                    "United States of America": "United States",
                    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
                    "China, Hong Kong Special Administrative Region": "Hong Kong"}, inplace=True)
Energy.reset_index(inplace=True)

Energy

,Country,Energy Supply,Energy Supply per capita,% Renewable
0,Afghanistan,3.210000e+08,10.0,78.6693
1,Albania,1.020000e+08,35.0,100
2,Algeria,1.959000e+09,51.0,0.55101
3,American Samoa,NaN,NaN,0.641026
4,Andorra,9.000000e+06,121.0,88.6957
...,...,...,...,...
222,Viet Nam,2.554000e+09,28.0,45.3215
223,Wallis and Futuna Islands,0.000000e+00,26.0,0
224,Yemen,3.440000e+08,13.0,0
225,Zambia,4.000000e+08,26.0,99.7147
